In [2]:
import sllm2
import torch
from torch import nn
from plotly import express as ex

In [3]:
with open("../simple-llm2/cache.txt") as f:
    db = f.read()

In [7]:
vocab = sorted(set(db))
vocab_size = vsz = len(vocab)
tokenizer = sllm2.tokenizer.Tokenizer()
tokenizer.train(vocab)

In [8]:
tokenizer.decode([0,1,2,3])

['\n', ' ', '"', '&']

In [9]:
data = torch.tensor(tokenizer.encode(db))
data.shape , data.dtype , data.device

(torch.Size([10000]), torch.int64, device(type='cpu'))

In [14]:
split = int (len(db) * 0.9)
train = data[:split]
test = data[split:]

In [19]:
"".join(tokenizer.decode(test.tolist()))

"s throughout the world.\nIn 1988, Irvine became the first non-American president of World Vision International.\nJim Daly.\ncame from Woodside army camp where he was a YMCA army officer.\nAfter his Army service he joined the Adelaide Y as Extension Secretary under Irvine having responsibility for developing boards, programmes and ensuring the viability of Walkerville, Northern Districts (Kilburn), West Croydon and Elizabeth Branches.\nHe was a senior leader at the first Kangaroo Island Camp and many others in preceding years.\nThey were initially organised initially by Don McCallum, Physical Education Director and in following years by Ross Baxter Glen Powell, Gary Kelly, Dean Manning, Dave Badger, Tim Looker.\nThe Kangaroo Island Camps ran for nearly 40 years.\nDaly's particular interest was in Adventure Camping.\nHe personally led groups of senior leaders on Outward Bound-type expeditions to New Zealand, Tasmania (Cradle Mountain Track, Flinders Ranges and the Grampians).\nDaly went

In [191]:
def get_data(batch_size , window_size = 20 , val = False):
    data = test if val else train
    random_pos = torch.randint(len(data)-window_size, (window_size,))
    x = torch.stack([ data[ i  :i+window_size  ] for i in random_pos ])
    y = torch.stack([ data[ i+1:i+window_size+1] for i in random_pos ])
    return x,y

x,y = get_data(batch_size=4, val=True)
print("\r"+"".join(tokenizer.decode(x[0].tolist())), end="")

ability of Walkervil

In [22]:
from dataclasses import dataclass

@dataclass
class ModelArg:
    vocab_size: int = -1
    emb_size: int = 32
    head_size: int = 32
    window_size: int = 20
    lr: float  = 1e-3
    batch_size: int = 32


In [196]:
model_args = ModelArg(
    vocab_size = vocab_size,
    emb_size = vocab_size,
    batch_size= 4,
    head_size= -1,
    window_size=20
)

class Transformer(nn.Module):
    def __init__(self , arg: ModelArg):
        super().__init__()
        self.arg = arg
        self.embedding = nn.Embedding(arg.vocab_size , arg.emb_size) # -> (batch , token , emb)
    
    def forward(self,x : torch.Tensor) -> torch.Tensor:
        b , t = x.shape
        logits = self.embedding(x)
        return logits

net = Transformer(model_args)
x,y = get_data(model_args.batch_size, model_args.window_size)
logits = net(x)

In [197]:
logits.shape

torch.Size([20, 20, 75])

In [207]:
l = nn.functional.softmax(logits , dim=-1)
torch.multinomial(l[:,-1,:], 1)

tensor([[56],
        [45],
        [53],
        [25],
        [53],
        [44],
        [ 0],
        [ 5],
        [ 4],
        [26],
        [29],
        [ 5],
        [47],
        [38],
        [24],
        [11],
        [19],
        [42],
        [27],
        [23]])